In [5]:
!pip install redis

    100% |████████████████████████████████| 71kB 264kB/s ta 0:00:01


In [6]:
import redis
import json 
import numpy as np
import pandas as pd
import pprint
from collections import Counter
#使用redis

In [7]:
r = redis.StrictRedis(host='redis', port=6379, db=0)
#使用連線 ip='redis' port=6379 db=0

In [8]:
with open('./jsondata/bigtable_1.2.json') as f:
    bigTable=json.load(f)
with open('./jsondata/final_tainan.json') as t:
    tainanData=json.load(t)
with open('./jsondata/TainanGoodDien_1.0.json') as t:
    goodDien=json.load(t)
#匯入各種資料
with open('./dict/MyDict.csv',encoding='big5') as t:
    mydict=t.read()
#自己的字典for同義字

In [9]:
synonyms=[{word:line.split(",")[1] for word in line.split(",")[1:]} for line in mydict.split("\n") if line.split(",")[0]!=""]
pinsynonymsdict={}
for synonym in synonyms:
    pinsynonymsdict.update(synonym)
del pinsynonymsdict[""]

pinsynonymsdict['飲料']='飲料專賣店'
pinsynonymsdict['喝的']='飲料專賣店'
pinsynonymsdict['涼的']='飲料專賣店'

pinsynonymsdict['綜子']='肉粽'

pinsynonymsdict['冰品']='剉冰、豆花'
pinsynonymsdict['冰']='剉冰、豆花'
pinsynonymsdict['豆花']='剉冰、豆花'
pinsynonymsdict['剉冰']='剉冰、豆花'

pinsynonymsdict['日式']='綜合日式料理'

# pinsynonymsdict['生魚片']='生魚片、壽司專賣'
# pinsynonymsdict['壽司']='生魚片、壽司專賣'

pinsynonymsdict['燒烤']='炭烤串燒'
pinsynonymsdict['串燒']='炭烤串燒'
pinsynonymsdict['串烤']='炭烤串燒'

pinsynonymsdict['下午茶']='西點烘焙、麵包'
pinsynonymsdict['點心']='西點烘焙、麵包'
pinsynonymsdict['甜點']='西點烘焙、麵包'

pinsynonymsdict
#取得同義字字典

{'CP值很低': '否划算',
 'CP值非常高': '划算',
 'CP值高份量夠': '划算',
 'Q彈': '食物美味',
 'Q彈柔嫩': '食物美味',
 'cp值不高': '否划算',
 'cp值太低': '否划算',
 'cp值很高': '划算',
 'hen好吃': '食物美味',
 'juicy': '食物美味',
 '一個人吃剛剛好': '小份量',
 '一口想把它喝光的衝動': '食物美味',
 '一品': '食物美味',
 '一問三不知': '否服務好',
 '一塵不染': '環境好',
 '一定要稱讚一下': '食物美味',
 '一樣無敵': '食物美味',
 '一致好評': '食物美味',
 '一般般': '否食物美味',
 '下午茶': '西點烘焙、麵包',
 '不乾不柴': '食物美味',
 '不乾柴': '食物美味',
 '不乾淨': '否環境好',
 '不划算': '否划算',
 '不卡牙': '食物美味',
 '不可多得': '服務好',
 '不夠酥脆': '否食物美味',
 '不太優': '否食物美味',
 '不太喜歡': '否食物美味',
 '不太好吃': '否食物美味',
 '不太愛吃': '否食物美味',
 '不太慣': '否食物美味',
 '不好吃': '否食物美味',
 '不對味': '否食物美味',
 '不怎麼樣': '否食物美味',
 '不想離開': '其他正面情緒',
 '不愛': '否食物美味',
 '不愛吃': '否食物美味',
 '不愛它': '否食物美味',
 '不搭': '否食物美味',
 '不敢恭維': '否環境好',
 '不斐': '否划算',
 '不新鮮': '否食物美味',
 '不會再來': '否其他正面情緒',
 '不會去第二次': '否其他正面情緒',
 '不會吃不起': '划算',
 '不會太重': '口味清淡',
 '不會太鹹': '口味清淡',
 '不會過於甜膩': '食物美味',
 '不會過甜': '食物美味',
 '不會過膩': '食物美味',
 '不油膩': '食物美味',
 '不熱': '否食物美味',
 '不用擔心': '其他正面情緒',
 '不用贅述就是好吃阿': '食物美味',
 '不算便宜': '否划算',
 '不脆': '否食物美味',
 '不膩口': '食物美

In [10]:
for i in r.keys():
    r.delete(i)
for word in pinsynonymsdict:
    if pinsynonymsdict[word]=='否食物美味':
        pinsynonymsdict[word]='食物不好吃'
    elif pinsynonymsdict[word]=='否划算':
        pinsynonymsdict[word]='價格較高昂'
    elif pinsynonymsdict[word]=='否環境好':
        pinsynonymsdict[word]='環境比較不好'
    elif pinsynonymsdict[word]=='否服務好':
        pinsynonymsdict[word]='服務較差'
    elif pinsynonymsdict[word]=='否其他正面情緒':
        pinsynonymsdict[word]='評論有些負面情緒詞'
#將否情緒改成tag用詞


pinsynonymsdict

{'CP值很低': '價格較高昂',
 'CP值非常高': '划算',
 'CP值高份量夠': '划算',
 'Q彈': '食物美味',
 'Q彈柔嫩': '食物美味',
 'cp值不高': '價格較高昂',
 'cp值太低': '價格較高昂',
 'cp值很高': '划算',
 'hen好吃': '食物美味',
 'juicy': '食物美味',
 '一個人吃剛剛好': '小份量',
 '一口想把它喝光的衝動': '食物美味',
 '一品': '食物美味',
 '一問三不知': '服務較差',
 '一塵不染': '環境好',
 '一定要稱讚一下': '食物美味',
 '一樣無敵': '食物美味',
 '一致好評': '食物美味',
 '一般般': '食物不好吃',
 '下午茶': '西點烘焙、麵包',
 '不乾不柴': '食物美味',
 '不乾柴': '食物美味',
 '不乾淨': '環境比較不好',
 '不划算': '價格較高昂',
 '不卡牙': '食物美味',
 '不可多得': '服務好',
 '不夠酥脆': '食物不好吃',
 '不太優': '食物不好吃',
 '不太喜歡': '食物不好吃',
 '不太好吃': '食物不好吃',
 '不太愛吃': '食物不好吃',
 '不太慣': '食物不好吃',
 '不好吃': '食物不好吃',
 '不對味': '食物不好吃',
 '不怎麼樣': '食物不好吃',
 '不想離開': '其他正面情緒',
 '不愛': '食物不好吃',
 '不愛吃': '食物不好吃',
 '不愛它': '食物不好吃',
 '不搭': '食物不好吃',
 '不敢恭維': '環境比較不好',
 '不斐': '價格較高昂',
 '不新鮮': '食物不好吃',
 '不會再來': '評論有些負面情緒詞',
 '不會去第二次': '評論有些負面情緒詞',
 '不會吃不起': '划算',
 '不會太重': '口味清淡',
 '不會太鹹': '口味清淡',
 '不會過於甜膩': '食物美味',
 '不會過甜': '食物美味',
 '不會過膩': '食物美味',
 '不油膩': '食物美味',
 '不熱': '食物不好吃',
 '不用擔心': '其他正面情緒',
 '不用贅述就是好吃阿': '食物美味',
 '不算便宜': '價格較高昂',
 '不脆': '

In [11]:
dienscore={dien['name']:round(dien['score'],2) for dien in bigTable}
dienscore
#kv >店:分數 排序用

{'同記安平豆花(安平總店)': 6.03,
 '深藍咖啡館': 8.32,
 '阿堂鹹粥': 8.13,
 '富盛號碗粿': 10.85,
 '依蕾特布丁奶酪(安平門市)': 8.95,
 '食下有約 想法廚房': 34.61,
 '飛饗創意西式料理': 35.99,
 '莉莉水果店': 8.73,
 '連得堂餅舖': 3.51,
 '矮仔成蝦仁飯': 9.43,
 '泰成水果店': 11.11,
 '騷烤家(前鋒總店)': 27.63,
 '阿松割包': 11.62,
 '秉醇烘焙坊': 18.17,
 '冰鄉': 7.53,
 '小銅鍋義式餐廳(德安誠品店)': 42.08,
 '周氏蝦捲(總店)': 10.43,
 '大菜市純薏仁': 9.68,
 '聯盈發點心專門店': 14.9,
 'OMAYA春川炒雞(台南永華店)': 28.46,
 '西井村蜂蜜滷味(安平店)': 17.22,
 '瑞益行': 16.92,
 '六千牛肉湯': 11.72,
 '文章牛肉湯': 10.33,
 '拿滋派瑞 Nuts Pirates': 17.75,
 '陳家蚵捲': 9.62,
 'EVA伊娃日式泡芙專賣店': 8.92,
 'A WEEK PANCAKE&COFFEE': 23.33,
 '邱家小卷米粉': 7.79,
 '阿裕現宰牛肉涮涮鍋': 13.62,
 '山根壽司': 13.76,
 '金得春捲': 11.48,
 '克林台包': 5.14,
 'Doga香酥脆椒': 6.59,
 '懷舊小棧': 7.05,
 'FUN Tower日式可麗餅': 20.93,
 '阿憨鹹粥': 8.32,
 '花見宇治抹茶甘味処': 18.36,
 '鹿早茶屋': 16.1,
 '窩吧': 19.67,
 'Kadoya喫茶店': 14.3,
 '王西勢食品行': 7.58,
 '窄門咖啡': 2.39,
 '秀治咖啡簡餐(臨安店)': 20.67,
 '橋北屋日本家庭料理(永福店)': 12.97,
 '歐加里(台南大學店)': 38.98,
 '華侖婷娜': 13.13,
 '阿明豬心': 12.11,
 '旗哥牛肉湯': 7.68,
 '裕成水果店': 4.78,
 '太陽牌冰品': 10.66,
 '赤崁棺材板': 12.36,
 '保哥黑輪': 6.18,
 '

In [12]:

dientag={dien['name']:set(dien['tags']) for dien in bigTable}
#kv >店:tagset
dientag

{'同記安平豆花(安平總店)': {'剉冰、豆花', '同記安平豆花', '安平', '服務較差', '珍珠', '紅豆', '豆花'},
 '深藍咖啡館': {'千層', '栗子', '派', '深藍', '蛋糕', '西點烘焙、麵包', '非常貴', '食物美味'},
 '阿堂鹹粥': {'價格較高昂', '其他小吃', '粥', '虱目', '阿堂', '食物美味', '魚腸', '鹹粥'},
 '富盛號碗粿': {'其他小吃', '安平', '富盛號', '小份量', '民族', '碗粿', '食物美味', '魚羹'},
 '依蕾特布丁奶酪(安平門市)': {'依蕾特', '典藏', '奶酪', '布丁', '抹茶', '西點烘焙、麵包', '食物美味'},
 '食下有約 想法廚房': {'其他正面情緒',
  '其他異國料理',
  '區郡',
  '想法廚房',
  '環境好',
  '花雕',
  '蹄膀',
  '非常好吃',
  '食下有約',
  '食物很香',
  '食物精緻漂亮'},
 '飛饗創意西式料理': {'其他正面情緒',
  '典藏',
  '土壤',
  '服務好',
  '沙拉',
  '燉',
  '環境超棒',
  '義式料理',
  '非常好吃',
  '食物精緻漂亮',
  '饗'},
 '莉莉水果店': {'價格較高昂', '剉冰、豆花', '前路', '水果', '肉圓', '莉莉', '豆花', '食物美味'},
 '連得堂餅舖': {'休閒零食', '價格較高昂', '兩包', '崇安', '煎餅', '連得堂', '餅家'},
 '矮仔成蝦仁飯': {'價格較高昂', '其他小吃', '矮仔成', '蛋湯', '蝦仁', '蝦仁飯', '食物美味', '鴨蛋'},
 '泰成水果店': {'價格較高昂', '哈密瓜', '哈蜜瓜', '果汁', '正興', '瓜瓜', '芒果', '食物美味'},
 '騷烤家(前鋒總店)': {'其他正面情緒', '府城騷烤家', '油條', '炭烤串燒', '燒烤', '甜不辣', '非常好吃', '騷烤'},
 '阿松割包': {'價格較高昂', '割包', '小份量', '瘦肉', '花生醬', '豬舌', '豬舌包', '食物美味', '麵食點心'},
 '秉醇烘焙坊':

In [13]:
pipin=[('食物美味','食物不好吃',"非常好吃",'非常不好吃'),('划算','價格較高昂',"非常便宜",'非常貴')
       ,('環境好','環境比較不好',"環境超棒",'環境很差'),('服務好','服務較差',"服務一級棒",'服務很差')
       ,('其他正面情緒','評論有些負面情緒詞',"網路評論正面情緒非常多",'網路評論負面情緒很多')]
for dien in dientag:
    for pin in pipin:
        if pin[2] in dientag[dien]:
            dientag[dien].add(pin[0])
        if pin[3] in dientag[dien]:
            dientag[dien].add(pin[1])
#將包含的情緒意義加入querytag
dientag

{'同記安平豆花(安平總店)': {'剉冰、豆花', '同記安平豆花', '安平', '服務較差', '珍珠', '紅豆', '豆花'},
 '深藍咖啡館': {'價格較高昂', '千層', '栗子', '派', '深藍', '蛋糕', '西點烘焙、麵包', '非常貴', '食物美味'},
 '阿堂鹹粥': {'價格較高昂', '其他小吃', '粥', '虱目', '阿堂', '食物美味', '魚腸', '鹹粥'},
 '富盛號碗粿': {'其他小吃', '安平', '富盛號', '小份量', '民族', '碗粿', '食物美味', '魚羹'},
 '依蕾特布丁奶酪(安平門市)': {'依蕾特', '典藏', '奶酪', '布丁', '抹茶', '西點烘焙、麵包', '食物美味'},
 '食下有約 想法廚房': {'其他正面情緒',
  '其他異國料理',
  '區郡',
  '想法廚房',
  '環境好',
  '花雕',
  '蹄膀',
  '非常好吃',
  '食下有約',
  '食物很香',
  '食物精緻漂亮',
  '食物美味'},
 '飛饗創意西式料理': {'其他正面情緒',
  '典藏',
  '土壤',
  '服務好',
  '沙拉',
  '燉',
  '環境好',
  '環境超棒',
  '義式料理',
  '非常好吃',
  '食物精緻漂亮',
  '食物美味',
  '饗'},
 '莉莉水果店': {'價格較高昂', '剉冰、豆花', '前路', '水果', '肉圓', '莉莉', '豆花', '食物美味'},
 '連得堂餅舖': {'休閒零食', '價格較高昂', '兩包', '崇安', '煎餅', '連得堂', '餅家'},
 '矮仔成蝦仁飯': {'價格較高昂', '其他小吃', '矮仔成', '蛋湯', '蝦仁', '蝦仁飯', '食物美味', '鴨蛋'},
 '泰成水果店': {'價格較高昂', '哈密瓜', '哈蜜瓜', '果汁', '正興', '瓜瓜', '芒果', '食物美味'},
 '騷烤家(前鋒總店)': {'其他正面情緒',
  '府城騷烤家',
  '油條',
  '炭烤串燒',
  '燒烤',
  '甜不辣',
  '非常好吃',
  '食物美味',
  '騷烤'},
 '阿松割包': {'價格較高昂', '割包

In [14]:
alltag=set()
for dien in dientag:
    alltag.update(dientag[dien])
alltag.remove("")
alltag
#找出所有tag

{'豬肉餅',
 '沾劫心',
 '鮮加',
 '驛站',
 '向光',
 '湯給',
 '蟹黃',
 '探險',
 '飛魚',
 '安和店',
 '還合味',
 '周子',
 '大眼',
 '請問',
 '新細',
 '助仔碗粿',
 '優格醬',
 '只賣',
 '金城',
 '白菜',
 '鍋說',
 '外設',
 '蜂巢',
 '拜託',
 '煙火',
 '田徑隊',
 '永吉',
 '瞎操心',
 '二次',
 '脆雞',
 '保華',
 '坎帕尼亞',
 '波提',
 '我及',
 '紅豆冰',
 '冰做',
 '真滴',
 '牛旁',
 '德元',
 '那罐',
 '累好',
 '賣場',
 '日朝食',
 '波菜',
 '還點',
 '平平安安',
 '王鎖店',
 '吊橋',
 '點因',
 '巴著',
 '鹿府',
 '咪',
 '洋房',
 '影月',
 '靜靜的',
 '菜二桌',
 '牠',
 '包後就',
 '糯夫',
 '郎',
 '陪要',
 '折合',
 '一點一滴',
 '玉竹',
 '三年',
 '小黃瓜',
 '小餅',
 '蒸氣',
 '九龍',
 '茶軒',
 '河內',
 '碼頭',
 '樂爸',
 '富霖',
 '酢醬麵',
 '反光板',
 '區知',
 '超悶',
 '附送',
 '加嫩',
 '食光啡宿',
 '萬香',
 '老舊',
 '高鐵',
 '手套',
 '下圍棋',
 '玉梅',
 '橙子',
 '凌波橋',
 '田山',
 '餃子',
 '福勝亭',
 '蠻嫩',
 '多餒',
 '忌廉',
 '喜慣',
 '慕絲',
 '功料',
 '專門店',
 '四段',
 '瑞吉兒',
 '重口味',
 '福來伯',
 '姨',
 '神農',
 '豬前',
 '代課',
 '鴨腳',
 '工藝',
 '兄長',
 '錫蘭',
 '三本',
 '有田原',
 '片鴨飯',
 '名模',
 '聚香園',
 '啄',
 '黑番',
 '美食街',
 '點最常',
 '和記',
 '正德',
 '文南',
 '沒話說',
 '根乾',
 '片成',
 '小炒',
 '請原諒',
 '單點',
 '九間',
 '用紅羅',
 '昌津柴魚',
 '姊夫',
 '丁骨',
 '雪點',
 '鮮榨',
 '漢',
 '紅髮

In [15]:
table_df=pd.DataFrame(bigTable)

In [16]:
pprint.pprint(dientag)

{'0.618': {'擂茶', '食物美味', '蒸蛋', '複合式', '熊貓', '煎餃', '刈包'},
 '0628全日早午餐店': {'份量大',
                '偏酸',
                '全日',
                '其他正面情緒',
                '司康',
                '小鐵',
                '烘蛋',
                '西式早餐',
                '貓咪',
                '食物精緻漂亮',
                '食物美味'},
 '1004 Brunch 早午餐': {'其他正面情緒',
                     '勾咕',
                     '天使',
                     '帕里尼',
                     '早午餐',
                     '服務好',
                     '環境好',
                     '複合式',
                     '韓國',
                     '食物美味'},
 '1010湘(台南三越西門店)': {'偏辣',
                    '價格較高昂',
                    '服務好',
                    '湖南菜',
                    '湘',
                    '湘菜',
                    '湘菜(湖南菜)',
                    '皮蛋',
                    '西米露',
                    '重口味',
                    '非常好吃',
                    '食物美味'},
 '108巷咖啡': {'早午餐', '價格較高昂', '迪克', '芬堡', '查爾斯', '藍赫法', '班尼'},
 '123冰城': {'四段',

                        '窯',
                        '義式料理',
                        '食物很香',
                        '食物美味'},
 'Pain Perdu 邦貝樂啾咖啡館': {'泰國', '勝博殿', '價格較高昂', '食物美味', '吉米', '複合式', '瓦城', '豬排'},
 "Perfetto Caff'e完美咖啡": {'完美', '鬆餅', '其他正面情緒', '複合式', '三明治', '咖啡', '小妹'},
 'Perfume Dance 跳舞香水(南紡門店)': {'價格較高昂',
                              '其他正面情緒',
                              '小馬',
                              '環境好',
                              '蜜糖',
                              '西點烘焙、麵包',
                              '跳舞',
                              '面具',
                              '食物很香',
                              '食物精緻漂亮',
                              '食物美味',
                              '香水'},
 'Pho越南河粉': {'青木瓜', '老母', '湯河粉', '越南', '其他異國料理', '河粉'},
 'Picturesque 塗鴉空間書店': {'價格較高昂',
                        '其他正面情緒',
                        '其它主題餐廳',
                        '口碑',
                        '塗鴉',
                        '書店',
                   

 '呷飽飽創意鍋燒麵(台南仁德店)': {'', '無評', '麵攤、麵店'},
 '咆嘯城市': {'有礙', '其他正面情緒', '爵士', '現場演奏餐廳', '飲酒', '過量', '小毛'},
 '和事饌蔥捲餅': {'', '無評', '蔥油餅、蔥抓餅'},
 '和喫': {'焙餅', '其他小吃', '私蜜', '鬆餅', '食物美味', '價格較高昂', '奶糖', '善化'},
 '和喫GRAZE手燒鬆餅': {'先探', '河馬', '西點烘焙、麵包', '大桔', '百事', '球現'},
 '和喫咖啡': {'胖卡', '鬆餅', '脆燒', '食物美味', '價格較高昂', '薄餅', '複合式', '佐脆燒'},
 '和恬咖啡': {'筱倩', '咖啡專賣', '坦尚', '環境好', '老闆娘', '咖啡', '扎片', '小份量'},
 '和意軒': {'國小側', '原福洱', '懷恩街', '無評', '茶棧', '中式', '順延'},
 '和意軒(原：福洱茶棧)': {'快給我', '叩', '環境比較不好', '素食', '中式', '萬聖節', '餐聽'},
 '和民居食屋(台南店)': {'古拉', '和民', '民居', '食屋', '價格較高昂', '食物美味', '其他日式料理', '大遠'},
 '和津壽司日本料理': {'', '無評', '生魚片、壽司專賣'},
 '和緯蝦仁飯': {'肉燥飯', '其他小吃', '食物美味', '價格較高昂', '蝦仁飯', '食物很香', '葉子', '目鏡', '魚湯'},
 '和興冰菓部': {'香蕉冰', '剉冰、豆花', '瀛', '冰店', '菁', '刨冰'},
 '和記鍋貼': {'和記', '環境比較不好', '鍋貼', '麵食點心', '蒸餃', '南園', '酸辣湯'},
 '和記鍋貼‧蒸餃': {'則必點', '和記鍋貼', '鍋貼', '店夠', '麵食點心', '不變'},
 '和野拉麵': {'食物美味', '八峰', '拉麵', '花月嵐', '日式麵食專賣', '一風堂', '骨家'},
 '和食日式廚房': {'酒駕', '蒸物', '醋物', '無評', '本店', '綜合日式料理', '條例'},
 '咔卡o將人文茶飲(北安總店)':

          '食物美味',
          '香魚'},
 '提卡異國料理': {'異國', '附湯', '附飯', '堤卡', '其他異國料理', '系列'},
 '提摩希歐式餐坊(台南東安店)': {'其他正面情緒',
                    '其他異國料理',
                    '划算',
                    '布蕾',
                    '提摩希',
                    '東安',
                    '焗',
                    '環境好',
                    '重口味',
                    '非常好吃',
                    '食物很香',
                    '食物精緻漂亮',
                    '食物美味',
                    '餐坊'},
 '揣茶': {'', '無評', '飲料專賣店'},
 '揪揪桑鐵板燒': {'鐵板', '價格較高昂', '魚露', '熱炒、快炒', '越南', '第一郡越南小館', '揪'},
 '揪港味': {'粵菜 港式飲茶', '價格較高昂', '咖哩', '香港', '魚蛋', '海安', '車仔'},
 '摩力沙牛排(麻豆店)': {'牛排', '無骨', '豬排', '夜市', '美式料理', '摩力', '服務好'},
 '摩啡館': {'極光', '胖拔', '柚汁', '芯芯', '複合式', '打上來', '偏辣'},
 '摩斯漢堡 MOS BURGER(台南健康店)': {'價格較高昂',
                            '堡',
                            '摩斯',
                            '漢堡、炸雞',
                            '環境比較不好',
                            '章魚',
                            '雙醬',
           

 '濰克早午餐': {'濰克', '划算', '早午餐', '食物美味', '脆薯', '食物很香', '份量大', '透亮', '潛艇', '服務好'},
 '濰克早午餐 Wake Up Brunch': {'雜糧', '漢堡', '濰克早午餐', '西式早餐', '拚盤', '滿高纖'},
 '濰克早午餐(台南崇善店)': {'', '無評', '早午餐'},
 '濰克早午餐-台南崇善店': {'份量大',
                 '其他正面情緒',
                 '崇善店',
                 '拼盤',
                 '早午餐',
                 '濰克',
                 '濰克早午餐',
                 '環境好',
                 '雜糧',
                 '非常好吃',
                 '食物美味'},
 '瀧來食坊': {'生魚片', '食物美味', '八角', '綜合日式料理', '海膽', '企鵝', '壽司'},
 '灣裡老兵意麵': {'不糊', '老兵', '食物美味', '爸比', '麵食點心', '媽咪', '肉燥'},
 '灣裡鴨肉焿': {'', '無評', '其他小吃'},
 '火力雞嘉義火雞肉飯': {'其他小吃', '自求多福', '運勢', '這請', '網絡', '圓融'},
 '火城麵店': {'火城', '灣裡路', '小善', '杜火城', '麵食點心', '母魚'},
 '火岩燒牛排': {'', '無評', '美式料理'},
 '火燄熱狗堡(台南中華店)': {'', '無評', '西點烘焙、麵包'},
 '火雞肉飯': {'肉伯', '價格較高昂', '嘉義', '葉子', '雞肉飯', '熱炒、快炒', '火雞肉'},
 '炒翻天(育樂店)': {'乙町翁', '翻天', '三鮮', '大甲', '炒飯', '台菜餐廳'},
 '炒飯專家': {'警察局', '蛋炒飯', '餓門', '蝦仁', '其它中式料理', '炒飯'},
 '炭吉168韓國料理': {'第二面', '銅盤', '環境比較不好', '韓式料理', '鯛魚', '

                '帕莎',
                '德國',
                '服務一級棒',
                '服務好',
                '法式料理',
                '環境好',
                '西餐廳',
                '豬腳',
                '非常好吃',
                '食物精緻漂亮',
                '食物美味'},
 '臻品香現烤蛋糕': {'食物美味', '西點烘焙、麵包', '冷藏', '品香', '蛋糕', '大盒', '臻', '服務好'},
 '臻好味食堂': {'', '無評', '其它中式料理'},
 '臻皇港式餐廳': {'中蝦',
            '偏甜',
            '叉燒',
            '港式',
            '燒鴨',
            '環境好',
            '粵菜 港式飲茶',
            '臻',
            '食物很香',
            '食物美味'},
 '興豆花': {'但豆', '店裡長', '剉冰、豆花', '綿濃稠豆', '元大', '豆花'},
 '舊來發餅舖': {'老灶', '西點烘焙、麵包', '暹羅', '木櫃', '糕餅', '黏土'},
 '舊公寓': {'禁菸', '酒吧', '價格較高昂', '昰', '調酒', '特調', '酒吧 Lounge Bar'},
 '舊振南餅店(台南店)': {'月餅', '鳳梨', '李白', '西點烘焙、麵包', '包裝', '振南'},
 '舊振南餅店(台南新光三越)': {'', '無評', '西點烘焙、麵包'},
 '舊永瑞珍喜餅': {'喜餅', '舊永瑞珍', '西點烘焙、麵包', '烏餡', '烏豆', '蓮蓉'},
 '舊烘爐': {'安平', '鬆餅', '老屋', '複合式', '海山', '兩百年'},
 '舒果新米蘭蔬食(台南府前店)': {'偏甜',
                    '價格較高昂',
                    '沙

 '麥克咖啡': {'小林', '因冰', '爆高', '複合式', '吊鐘', '咖啡'},
 '麥午茶(麥午茶)': {'鬆餅', '食物精緻漂亮', '咖啡專賣', '麥午', '魚起司', '鮪', '腸堡'},
 '麥叔叔大阪燒': {'文字燒、大阪燒', '燒的', '環境比較不好', '講可', '糊美奶', '大阪', '片照'},
 '麥味登 Cafe&Brunch(大仁店)': {'', '無評', '西式早餐'},
 '麥味登 Cafe&Brunch(東橋店)': {'吐司', '麥味', '排擠', '柔嫩', '整齊劃一', '西式早餐'},
 '麥味登 Cafe&Brunch(正南店)': {'勞的', '蛋裡面', '一吃成', '比麥', '西式早餐', '明日'},
 '麥味登 Cafe&Brunch(長榮店)': {'分響', '麥味', '早餐', '登', '略差', '西式早餐'},
 '麥基香雞排': {'丸子', '其他小吃', '花枝', '丸', '雞肉', '善化'},
 '麥多古堡童話餐廳': {'布蕾', '麥多', '食物精緻漂亮', '熊熊', '食物美味', '小燕子', '複合式', '可愛', '服務好'},
 '麥布丁蛋糕 My Pudding Cake': {'麥', '價格較高昂', '西點烘焙、麵包', '蛋糕', '不吸睛', '稱佳', '布丁'},
 '麥果城市行動鮮茶': {'橙子', '飲料專賣店', '燈箱', '霸氣', '蔬果', '獨家'},
 '麥樂比而-永華店': {'', '無評', '啤酒屋'},
 '麥當勞(台南中華二)': {'', '無評', '漢堡、炸雞'},
 '麥當勞(台南中華店)': {'蘋果派', '派', '香橙', '一怒之下', '麥克', '漢堡、炸雞'},
 '麥當勞(台南仁德店)': {'彎曲', '鬆餅', '薯條', '奇士', '麥當勞', '漢堡、炸雞'},
 '麥當勞(台南大學店)': {'', '無評', '漢堡、炸雞'},
 '麥當勞(台南新營店)': {'價格較高昂', '冰炫風', '碧玉', '冰炫', '抹茶', '樂樂', '偏甜', '漢堡、炸雞'},
 '麥當勞(台南東門店)': {'經理', '麥當勞', '小

In [17]:
a=table_df.tags
tagdien=[]
for i in a:
    tagdien+=i
tagdien={tag:set() for tag in set(alltag)}
for dien in dientag:
    for tag in dientag[dien]:
        if tag!="":
            tagdien[tag].add(dien)
#kv >tag:店set
pprint.pprint(tagdien)

{'一下場': {'雅+達藤原豆腐冰'},
 '一中': {'一中街 大輪 紅豆餅'},
 '一串': {'一串心燒烤屋', '泰式幽靈串燒', '串燒心小炒'},
 '一九九八年': {'美樂蛋糕(美樂蛋糕)'},
 '一事': {'FAB義式料理'},
 '一二五': {'二師兄古早滷味'},
 '一二層': {'松笠屋'},
 '一佰年': {'龍泉食堂'},
 '一保堂': {'大菜市純薏仁'},
 '一個紙': {'自來香餡餅(新營店)'},
 '一個肚子': {'村伯菜粽'},
 '一元': {'仁弘宴席', '繆思義式餐坊(原繆思園)'},
 '一到': {'月芽麵包'},
 '一則以喜': {'三皇三家(台南新光三越門市)'},
 '一則以憂': {'三皇三家(台南新光三越門市)'},
 '一剎那': {'安迪的店'},
 '一包': {'啾甘心濃郁養生茶飲', '林永泰興蜜餞行', '倍樂烘焙坊', '東東鮮蝦餅'},
 '一匹': {'一代茶師'},
 '一吃成': {'麥味登 Cafe&Brunch(正南店)'},
 '一味': {'一味品碗粿'},
 '一啟': {'歡樂牛排'},
 '一大家子': {'異人館咖啡部屋(中華店)'},
 '一大截': {'有夠香火雞肉飯'},
 '一大養生麵食': {'味聖豆花'},
 '一夫': {'淳鳩一夫拉麵'},
 '一夾取': {'慈濟園素食'},
 '一對一': {'千尋ちひろ日式餐飲'},
 '一店': {'瑞穗沙茶爐(一店)'},
 '一心': {'林家素食', '一心二葉創意料理部館'},
 '一怒之下': {'麥當勞(台南中華店)'},
 '一恒': {'海之味海產'},
 '一成': {'Double Cheese 披薩吃到飽餐廳'},
 '一手交錢': {'咖哩の蛋蛋'},
 '一手私藏世界紅茶': {'一手私藏世界紅茶(台南東寧店)'},
 '一批': {'日式壽司 惠比壽'},
 '一拿結': {'早鳥手作三明治'},
 '一斤': {'韓氏古早味油飯'},
 '一方': {'一方日朝食X喫茶'},
 '一景': {'田田茶舖(台南店)'},
 '一朗': {'ICHIRO，ㄧ起囉！'},
 '一朝一夕': {'大埔平價鐵板燒(立人店)'},
 '一本': {'一本花田(南門店)'

        '鮮魚蔥肉粿',
        '黃記古早包子饅頭店(歸仁店)',
        '黑橋牌香腸肉燥飯',
        '鼎富臭豆腐'},
 '五十一': {'51 STREET 日式炸豬排 和洋料理', '51街日式炸豬排和洋料理'},
 '五十塊': {'陽記脆支'},
 '五十鈴': {'食在有藝'},
 '五吉': {'五吉堂麵包店'},
 '五吉堂': {'五吉堂麵包店'},
 '五品': {'Café & Meal MUJI 無印良品(台南西門店)'},
 '五妃': {'21世紀風味館(台南仁德家福店)',
        '三商巧福(台南仁德店)',
        '好麵道拉麵工坊',
        '小莫里咖啡輕食簡餐',
        '巴蜀風道地四川麻辣美食',
        '懷舊小棧',
        '找朋友吃雞蛋糕',
        '明仁鍋貼',
        '爭鮮迴轉壽司 SUSHI EXPRESS(仁德店)',
        '美都麵食店',
        '舞飛日式燒肉'},
 '五嬸': {'五嬸原味活海鮮'},
 '五子': {'阿勇家外燴辦桌'},
 '五子棋': {'耕讀園書香茶坊(永華店)'},
 '五店': {'鷗元氣爹斯嘎脆皮炸雞'},
 '五折': {'Wa食尚鍋燒', '東大門韓國料理館'},
 '五更': {'董娘豬腳', '陳媽香辣棧'},
 '五月天': {'裕益碗粿王'},
 '五期': {'吃對顏色醬匠、便當'},
 '五木': {'五木麻辣燙'},
 '五朵': {'泰航泰式主題餐廳(台南店)'},
 '五條': {'布列塔尼義大利麵餐廳'},
 '五榖': {'Starbucks Coffee 統一星巴克(南科門市)', '黃記古早饅頭肉包(長榮店)'},
 '五段': {'清代牛肉麵'},
 '五福': {'五福園溫泉美食館(關子嶺)'},
 '五穀': {'自然主義複合式餐飲', '梅子蔬食', '甜在心咖啡'},
 '五穀雜糧': {'薩朵 Sardoine'},
 '五色': {'曉食館台式蓋飯專門店'},
 '五花': {'七輪燒肉本舖',
        '不專精匠',
        '溫野菜-台南大遠百公園店',
        '焼肉ショ

 '值星官': {'寬心園(臺南永華店)'},
 '值極': {'咖啡深藏(台南裕農門市)'},
 '偏側': {'原味廚坊'},
 '偏像海': {'許家堡杏仁茶杏仁豆腐'},
 '偏吃料': {'劉家酸白菜火鍋(台南中華東路店)'},
 '偏淡且': {'招牌大滷麵'},
 '偏甜': {'3A同學會',
        '8%ice 冰淇淋專門店(新光三台南小西門門市)',
        '85度C咖啡蛋糕烘焙專賣店(永康中華店)',
        'Bungy Jump Korean BBQ 笨豬跳 - 台南店',
        'ChuChu Pâtisserie 法式甜點',
        'Cold Stone酷聖石頂級冰淇淋(台南南新門市)',
        'DACOZ! 大口吃',
        'ISABELLE 伊莎貝爾(台南東寧門市)',
        'IZZY CAFE',
        'JOE BAR酒吧',
        'Love Ice Cream 造型馬卡龍冰淇淋',
        'Mascot Bar  哥德式教堂',
        'Micasa Recipe 溫馨小窩',
        'Mr.Wheat 小麥先生創意料理(文平義式館)',
        'O卡桑 の 裸食代',
        'T&F 手作吐司',
        "mini Fen's 棻園3店",
        '一等涼冬瓜茶',
        '三皇三家(中華店)',
        '丹丹漢堡',
        '丹丹漢堡(仁德店)',
        '丹丹漢堡(安平店)',
        '九分十刻',
        '五條港府城伴手禮',
        '亞米廚房',
        '人人好冰菓室',
        '仙玉甜食',
        '佳湘西點烘培店',
        '倉庫早頓',
        '全量古早味紅茶店',
        '劉家楊桃湯',
        '功夫啦哩',
        '千喜火鍋館',
        '半個鍋個人火烤兩吃鍋物(台南海安店)',
        '南方米造秈稻碗粿',
        '南興鱔魚意麵',
        '台

           '阿財點心',
           '阿貴黑糖粉粿冰',
           '阿賓牛肉湯',
           '阿輝 炒鱔魚(西門圓環店)',
           '阿進鱔魚意麵',
           '阿銘牛肉湯',
           '阿銘牛肉麵',
           '阿銘鱔魚意麵',
           '阿隆黑輪',
           '阿霞飯店',
           '阿魯香腸熟肉',
           '阿鳳浮水虱目魚焿',
           '阿鳳碗粿',
           '阿龍意麵',
           '阿龍杏仁茶',
           '陳Q古早黑砂糖剉冰',
           '陳媽香辣棧',
           '陳家汕頭意麵',
           '陳家蚵捲',
           '陳氏蚵捲',
           '陳記真正紅燒土魠魚羹',
           '陶林日式涮涮鍋(台南永華店)',
           '陶湘涮涮鍋',
           '陽春麵',
           '集品蝦仁飯',
           '雙全紅茶',
           '雙雞牌豆油-萬味香醬園',
           '雞排王國',
           '霸味薑母鴨',
           '霸味薑母鴨(台南中華店)',
           '非常泰',
           '韋家乾麵',
           '韓國先生(OBBA店)',
           '韓巢炸雞(善化南科店)',
           '韓氏古早味油飯',
           '韓金婆婆豆腐酪(安平店)',
           '韓金婆婆豆腐酪(赤崁店)',
           '韓金婆婆食品行(育賢店)',
           '響亮冰舖',
           '順天冰棒',
           '順風號',
           '頡寶牛肉麵',
           '顏舍鍋物',
           '飄香養生飯糰',
           '飛行程是航空主題餐廳',
           '食光啡宿 Time Goes Fas

          '橋味屋甘草芭樂',
          '永林牛肉火鍋',
          '泰式香蕉煎餅-台南永康店',
          '洪媽鍋貼專賣店',
          '清香羊肉',
          '湯家蝦捲',
          '溫記手作麵',
          '烘廚(大林店)',
          '焗米G-me 焗烤壽司',
          '牧大畜牧場',
          '狗啃玉米',
          '甜心有機雞蛋',
          '皇品伴手禮 澎湖干貝醬',
          '目鏡蝦仁飯',
          '福神町拉麵燒(台南花園夜市)',
          '福豆屋(德安店)',
          '老鄉長景觀餐廳',
          '肉皮張魯味(金華店)',
          '良辰即食(仁德店)',
          '艸疏田木',
          '蔡家蚵嗲',
          '虎山鮮乳',
          '蠔記燒臘港式茶餐廳',
          '西肯廚房精品食堂',
          '赤崁堂rhalife',
          '輕食物語',
          '郭媽碳烤三明治',
          '金科威補天鮮滴雞湯',
          '金龍檳榔(裕豐店)',
          '阿春蛋包飯',
          '阿水伯包子',
          '陳記烤雞專家',
          '雙角號',
          '雙雞牌豆油-萬味香醬園',
          '雞巢日式炸雞',
          '領鮮水果專賣店',
          '首戶牛排(勇大店)',
          '首戶牛排(永大店)',
          '魚之家',
          '鳥居有菜輕鬆蔬食',
          '鴨霸當歸鴨',
          '麻善橋'},
 '其他速食': {'BONJOUR帕里尼 輕食',
          'SUBWAY(台南成大店)',
          '不只是沙拉',
          '大成集團',
          '松笠屋',
          '營養

...

 '臭味': {'獨臭之家'},
 '臭度': {'獨臭之家'},
 '臭暈': {'男子漢餡餅'},
 '臭氣沖天': {'阿埔哩臭豆腐'},
 '臭臭': {'21金小火鍋臭臭鍋(鹽水店)',
        '台南飯店港式飲茶',
        '大呼過癮臭臭鍋(新市店)',
        '旺角臭豆腐',
        '珍好味臭臭鍋',
        '辣媽臭臭鍋(仁德店)',
        '鍋寶臭臭鍋(武聖店)'},
 '臭豆腐': {'CHEFFRESH',
         '一派胡塩酵素臭豆腐',
         '一派胡塩酵素臭豆腐(大灣店)',
         '一派胡塩酵素臭豆腐(崇義店)',
         '一派胡塩酵素臭豆腐(新市店)',
         '九六雞蛋臭豆腐專賣',
         '儔儔臭豆腐',
         '哈哈哈脆皮臭豆腐',
         '姑婆麵',
         '學甲臭豆腐',
         '官記臭豆腐',
         '家閎養生臭豆腐',
         '小不點臭豆腐',
         '小不點臭豆腐專賣店',
         '張記涼麵',
         '彰化脆皮肉圓',
         '新市土地公臭豆腐',
         '旺角臭豆腐',
         '昆伯臭豆腐',
         '林家臭豆腐',
         '津華火鍋料理',
         '無名臭豆腐',
         '獨臭之家',
         '福來伯臭豆腐',
         '老江南臭豆腐',
         '蠔記燒腊茶餐廳',
         '豪記臭豆腐(台南旗艦店)',
         '郭媽媽臭豆腐',
         '鄉耕黃金臭豆腐',
         '金食堂',
         '阿埔哩臭豆腐',
         '阿太伯臭豆腐',
         '阿嬤臭豆腐',
         '阿彌專業臭豆腐',
         '阿松臭豆腐',
         '陳媽香辣棧',
         '顏舍鍋物',
         '香格格祕製臭豆腐',
         '香淳臭豆腐',
       

 '蛤蜊': {'布老虎廚坊', '蚵男鮮蚵本舖(鹽行夜市)', '吉祥叉燒麵屋'},
 '蛤蠣': {'錢源日式涮涮鍋(台南永康永大店)', '義式意適'},
 '蛸': {'蛸屋本舖'},
 '蜀': {'川蜀 四川麻辣燙', '蜀姥香麻辣鍋(台南府前店)'},
 '蜀姥香': {'咕嚕咕嚕', '蜀姥香麻辣鍋(台南府前店)', '蜀姥香麻辣鍋 永康店', '台南張記麻辣火鍋', '署姥香麻辣鍋(府前店)'},
 '蜂巢': {'餓魚咬冰', '果子餅乾果汁專賣店'},
 '蜂炮': {'阿桐意麵', '林家紅豆餅', '昆伯臭豆腐', '水煎包', '阿三意麵', '角樓', '鹽水煎包', '銀鋒冰果室', '阿姬意麵'},
 '蜂蛹': {'忘憂谷'},
 '蜂蜜': {'Dessert around US',
        'Forest Yogurt 森霖優格',
        '佳湘西點烘培',
        '西井村蜂蜜滷味(安中店)',
        '西井村蜂蜜滷味(安平店)',
        '長生本舖(北門店)'},
 '蜂蜜水': {'小葉鍋物料理'},
 '蜆': {'生機炁能滷肉飯黃金帝王蜆', '劉家楊桃湯', '東石日式料理'},
 '蜆湯': {'生機炁能滷肉飯黃金帝王蜆'},
 '蜘蛛人': {'大夯牛排館'},
 '蜜丸子': {'暖暖 麻辣 手作關東煮 碳烤'},
 '蜜嚕': {'壽賀喜屋(台南三越)'},
 '蜜塔': {'i.mita 愛米塔'},
 '蜜意': {'麻吉格'},
 '蜜是': {'壽賀喜屋(台南站前店)'},
 '蜜桃': {'蜜桃香冷飲店', '小澤銅鑼燒', '醋頭家'},
 '蜜梨': {'有時甘杯'},
 '蜜檸': {'繼光香香雞(台南南方公園)', '繼光香香雞(南紡夢時代店)'},
 '蜜汁': {'力都鐵板燒', '華師傅平價燒臘便當'},
 '蜜糖': {'17 cafe',
        '3A同學會',
        'BOG TEA ROOM',
        'CHADOR',
        'Cocobear烘焙坊',
        'Dazzling Cafe(台南新光三越中山店)',
        'Double Yummy',
        

 '部粉': {'歡樂牛排'},
 '部隊': {'OMAYA 春川炒雞(台南永康店)',
        'OMAYA春川炒雞(台南永華店)',
        '喆發樂樂鍋',
        '喆發樂樂鍋(金華店)',
        '大長今韓風館',
        '安平正統韓國料理',
        '老韓家韓式廚房',
        '芳鄰小館',
        '韓屋 韓式廚房',
        '韓屋韓式廚房',
        '韓朝韓式料理',
        '韓湘辣年糕',
        '韓鍋人(永康店)',
        '體驗分享・府城】♥愛體驗♥ OMAYA春川炒雞韓式料理-台南永康店'},
 '部館': {'一心二葉創意料理部館'},
 '郭': {'郭 綠豆湯(裕農店)'},
 '郭元益': {'郭元益(東寧門市)'},
 '郭先生': {'伍零壹'},
 '郭南成': {'6018 TEA & COFFEE(台南佳里店)'},
 '郭姐': {'郭姐茶坊'},
 '郭泓志': {'米糕專門'},
 '郭芃': {'小西腳碗粿'},
 '郭記': {'郭記肉粽'},
 '郵筒': {'貳拾捌號倉'},
 '鄉大': {'紅風車餐廳'},
 '鄉村': {'Happy Bear 花草鄉村廚房', '綠色法藍絲'},
 '鄉為': {'南都石頭鄉玉米'},
 '鄉野': {'鄉野炭燒羊肉爐', '鄉野手工拉麵', '鄉野羊肉爐'},
 '鄉野炭燒羊肉爐': {'鄉野羊肉爐'},
 '鄉長': {'老鄉長景觀餐廳'},
 '鄉間': {'Recycle 鄉間烘焙茶點坊'},
 '鄒孟翰': {'夏林汕頭意麵'},
 '鄧': {'老鄧牛肉麵'},
 '鄧麗君': {'珍品美食料理(佳里店)'},
 '鄭': {'尋秘桑田'},
 '鄭守柔': {'慶中街綠豆湯'},
 '鄭小姐': {'良食草堂'},
 '鄭成功': {'安平蚵仔煎'},
 '鄭文祈': {'鄭記黑珍珠玉米'},
 '鄭記': {'鄭記葱肉餅'},
 '鄭記黑珍珠玉米': {'永和商號'},
 '鄰近地區': {'欣福龍海鮮餐廳'},
 '酌收': {'加比咖啡'},
 '配料': {'西門粉圓冰店(西門店)', '儒家紅豆湯花生仁湯'},
 '配湯':

          '台南蔡家古早味肉粽',
          '台南雲象泰式餐廳',
          '台灣茶渠(崇善店)',
          '台灣雷夢',
          '合力歐印度料理',
          '吉品家手作烤餅',
          '吉田小火鍋-鍋燒麵',
          '名坂奇',
          '名牌手工包子',
          '名風現烤蛋糕',
          '味味香麵食店',
          '和緯蝦仁飯',
          '咕咕布朗早午餐',
          '咖哩本舖',
          '咖啡平方隨行吧(崇德店)',
          '咖啡平方隨行吧(東安店)',
          '咩 灣裡羊肉(保安店)',
          '品元扁食水餃',
          '哞王 原味炭烤牛排',
          '唇義義大利餐酒館',
          '唐媽媽早餐店',
          '喫茶初梅',
          '嘀家 DiJia',
          '圓竹發小籠湯包',
          '土匪烘焙事務所',
          '塔彼Q & Top-Q',
          '夏慕尼新香榭鐵板燒(永華店)',
          '多一點咖啡館 A Little More(台南美學館)',
          '多那之(台南崇德門市)',
          '大城家鄉菜',
          '大胖進焢肉飯',
          '大阪王將餃子專賣店(台南新光三越西門店)',
          '天下大飯店',
          '天心岩蔬食餐廳',
          '天然葉(北門店)',
          '奇奇亞義式手工冰淇淋',
          '奧尻爐端燒',
          '奶控',
          '好口味川粵館(中華西路店)',
          '好味乾拌麵',
          '威葳手作烘焙',
          '宇治88',
          '安平小舖',
          '安平陳記高粱黑豬肉香腸',
          '寬窄巷子四川麻辣鍋',
       

 '香佛': {'朵菈義大利麵', '蕎琪義大利麵'},
 '香台': {'東香台菜海產', '紜楓烤肉刈包'},
 '香和彈': {'飯冰冰餐飲屋'},
 '香圃': {'香圃麵店'},
 '香坊': {'香格里拉台南遠東大飯店-品香坊'},
 '香城': {'香城手扒雞(原：東帝士百貨)'},
 '香多': {'帝王食補紅面薑母鴨(東門二店)'},
 '香客': {'食香客雞會站雞排專賣(北門總店)', '雞排推薦-食香客雞會站雞排加盟(台南永康中山店)', '食香客雞會站(南科大店)'},
 '香干': {'小赤佬 干鍋店'},
 '香引人': {'黃記古早饅頭包子(公園店)'},
 '香怕': {'Kali Curry'},
 '香料': {'合力歐印度料理'},
 '香格里拉': {'香格里拉台南遠東大飯店-品香坊',
          '香格里拉台南遠東大飯店-大廳茶軒',
          '香格里拉台南遠東大飯店-遠東Cafe',
          '香格里拉台南遠東大飯店-醉月樓'},
 '香梅': {'異人館無國界料理(安南店)', '異人館咖啡部屋(中華店)', '一畝田園生活美食'},
 '香椿': {'栗味齋', '麒珍素食(善化店)'},
 '香橙': {'麥當勞(台南中華店)', '呷百二自然洋菓子'},
 '香檳': {'金剛の饌', '官記臭豆腐', 'AngelTalk法國軟麵包'},
 '香檸雞': {'遇見幸福異國創意料理'},
 '香水': {'Perfume Dance 跳舞香水(南紡門店)'},
 '香港': {'Cafe Alchemy 阿爾',
        'FAT Cat DELI',
        'Hej Nordic北歐日常',
        'TJB茶餐室(Carrefour台南安平店)(Carrefour台南安平店)',
        '九龍茶餐廳',
        '佬香港 港式餐廳',
        '十年.食堂 Ten Years',
        '台南 秘氏咖啡',
        '小香港茶餐廳',
        '捌伍貳冰室',
        '揪港味',
        '星仔冰室',
        '朱記香港茶水灘',
        '榮寓冰室',
    

In [18]:
len(tagdien['食物美味'])

1588

In [19]:
#r.ltrim('tutorial-list',0,-1000000)刪除list的方法(最後100000筆)
for dien in dientag:
    r.ltrim(dien,0,-1000000)
for tag in tagdien:
    r.ltrim(tag,0,-1000000)
#先刪除以免重覆加入
    
    
#將dientag跟tagdien分別加入redislist
for dien in dientag:
    r.lpush(dien,*dientag[dien])
for tag in tagdien:
    r.lpush(tag,*tagdien[tag])

In [32]:
#1用屬性找店家
querytags=input("請輸入想要的屬性(可輸入多個，以空白分開):")
if querytags!="":
    dienList=[]
    
    #將輸入的string變成taglist
    for query in set(querytags.split(" ")):
        print(query)
        #將情緒同義字對應query
        if query in pinsynonymsdict:
            query=pinsynonymsdict[query]
            
        #一個query可以找到一個店家list
        if len([i.decode() for i in r.lrange(query,0,10000)])==0:
            for tag in alltag:
                if query in tag:
                    query=tag
                    break
        print("實際tag："+query)
        if dienList==[]:
            dienList =[i.decode() for i in r.lrange(query,0,10000)]
        else:
        #多個query可以找到不同店家list得交集
            dienList = set(dienList) & set([i.decode() for i in r.lrange(query,0,10000)])
#     print('屬性搜索項目:')
#     print(dienList)

#2用店名找店家
querydeins=input("查店名:")
if querydeins!="":
    nameList=[]
    for dein in set(querydeins.split(" ")):
        print(dein)
        if nameList==[]:
            nameList =[j for j in [i.decode() for i in r.keys()]  if dein in j]
        else:
            nameList = set(nameList) & set([j for j in [i.decode() for i in r.keys()]  if dein in j])
#     print('店名搜索項目:')
#     print(nameList)
    
if querytags!="" and querydeins!="":
    allquery=set(dienList) & set(nameList)
elif querytags!="" and querydeins=="":
    allquery=set(dienList)
elif querytags=="" and querydeins!="":
    allquery=set(nameList)
    
print('綜合搜索項目:')
#     print(allquery)
queryscore=Counter({dien:dienscore[dien] for dien in dienscore if dien in allquery}).most_common(20)
pprint.pprint(queryscore)


請輸入想要的屬性(可輸入多個，以空白分開):好吃 美味 份量大
美味
實際tag：食物美味
好吃
實際tag：食物美味
份量大
實際tag：份量大
查店名:
綜合搜索項目:
[('洋城義大利餐廳', 53.34),
 ('格樓主題餐廳-Grimm House', 49.56),
 ('洁沐創義料理', 46.75),
 ('尊客牛排工坊', 46.5),
 ('MAP LAB 健康低脂創意早午餐沙拉', 45.82),
 ('鮨次郎壽司專賣店', 43.69),
 ('濰克早午餐', 43.0),
 ('福豆屋(德安店)', 42.82),
 ('不老蒸氣養生料理', 42.63),
 ('山代目酒藏', 41.91),
 ('八老爺吃飽喝足日式家庭料理', 41.44),
 ('陶林日式涮涮鍋', 41.1),
 ('郭師傅飯鍋料理', 40.07),
 ('愛咖客咖啡館', 39.56),
 ('青森Dryad時尚餐廳', 39.44),
 ('纓風壽司', 38.47),
 ('幸福湯味', 37.6),
 ('Takeat 輕享原食', 36.13),
 ('Curry+ Kitchen 誠欣廚房', 35.71),
 ('老恆春碳烤店', 35.27)]


In [30]:

for i in bigTable:
    if i['name'] in dict(queryscore).keys():
        print(i['name'])
        print(i['style'])
        print(i['score'])
        print(i['introduction'])
        print(i['featurescores'])
#         print(i['tags'])
        print('-'*50)
#         print(i['name'])

瑪莉洋房 Marie's House
法式料理
45.65217391304349
瑪莉洋房將頂級進口食材與在地鮮蔬搭配出歐陸料理，從前菜至甜點皆由主廚團隊細心研發手工製作。洋房裡有許多古董家具、燈飾與畫，打造出法國貴族居所般的雅致浪漫空間，希望顧客在用餐中讓美食領路，享受台南與歐洲融合交會的美好時光。
{'食物美味': 1.77, '食物很香': 0.95, '划算': -0.31, '環境好': 1.49, '服務好': 1.02, '食物精緻漂亮': 0.91, '其他正面情緒': 0.8, '份量大': 0.15, '重口味': 0.5, '偏辣': 0.06, '偏甜': 0.15, '小份量': 0.23, '口味清淡': 0.07}
--------------------------------------------------
天心岩蔬食餐廳
養生蔬食餐廳
50.766666666666715
【天心岩蔬食餐廳】於1981年成立，鄰近台南市政府，長期用心研發獨家蔬食料理，贏得廣大顧客喜愛。如今與在地有機小農合作，採購當季有機蔬食，搭配於現有菜單。餐廳並選用成本高於一般油品約4倍的椰子油做料理，確保餐點美味又安心。平日利用剩餘的新鮮果皮菜葉，回收製作成環保酵素，用來清洗蔬菜，不僅清洗安心，又能有效去除農藥殘留。【天心岩蔬食餐廳】服務方式為蔬食現點吃到飽，每人500元+10%，多元菜色，融入台南在地特色小吃，輔以泰式創意菜色，讓饕客體驗高貴不貴的蔬食饗宴。
{'食物美味': 1.85, '食物很香': 0.88, '划算': -0.05, '環境好': 1.9, '食物精緻漂亮': 1.23, '其他正面情緒': 1.14, '份量大': 0.47, '小份量': 1.15, '重口味': 1.0, '服務好': 0.49, '偏甜': 0.36, '偏辣': 0.09, '口味清淡': 0.2}
--------------------------------------------------
唇義義大利餐酒館
義式料理
49.788235294117605
我們是最接近義大利的餐酒館沒有華麗擺盤，不做精緻的包裝只追求真正道地風味的義大利菜我們的餐點帶一點隨興，但絕不隨便我們的味道比較清淡，因為講求是食物原味就像義大利人喜歡的家常菜單純、簡單、新

In [327]:
a=['d', 'c', 'b', 'a', 'MySQL', 'Mangodb', 'Redis', 'MySQL', 'Mangodb', 'Redis',"笨蛋"]
b=["笨蛋", 'Redis','yayaya']
list(set(a) & set(b)) #交集

['Redis', '笨蛋']

In [328]:
list(set(a) | set(b)) #聯集

['yayaya', 'Mangodb', 'b', 'a', 'c', 'MySQL', 'd', 'Redis', '笨蛋']

In [329]:
#list包函於list2 判斷的方法
set(['d', 'c', 'b']) & set(a)==set(['d', 'c', 'b'])

True

In [330]:
x='川品鐵板'
for i in bigTable:
    if x in i["name"]:
        print(i['tags'])
        print(i['score'])

['份量大', '海陸', '環境好', '非常好吃', '食物很香', '其他正面情緒', '鐵板', '划算', '川品', '鐵板燒', '套餐', '其它中式料理']
34.533333333333346
